# Create JPegImages folder and populate it with .jpg versions of the .tiff files

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import tifffile as tiff
import os
import PIL
from PIL import Image

The direction `../data` should contain links to `/data/dstl/three_band/` as `dstl` and to `/data/VOCdevkit2007/VOC2007/` as `voc` (but that's not used in this notebook).

In [12]:
data_dir = os.path.join(os.getcwd(), '..', 'data', 'dstl')
tiff_loc = os.path.join(data_dir, 'TIFFImages')
jpeg_loc = os.path.join(data_dir, 'JPegImages')
print data_dir

/home/ubuntu/dstl/notebooks/../data/dstl


In [13]:
def convert(filename):
    print filename
    im = tiff.imread(os.path.join(tiff_loc, filename + '.tiff'))
    im>>= 3 # downsample to 8 bits per pixel
    im_rgb = Image.fromarray(im.astype(np.uint8).transpose((1, 2, 0)))
    im_rgb.save(os.path.join(jpeg_loc, filename + '.jpg'))

In [15]:
for fn in os.listdir(tiff_loc):
    if os.path.isfile(os.path.join(tiff_loc, fn)):
        filename, file_extension = os.path.splitext(fn)

        if file_extension == '.tiff' and not os.path.isfile(os.path.join(jpeg_loc, filename + '.jpg')):
            convert(filename)